In [ ]:
# cd C:\Users\nguye\Downloads\PECC2\confluent-mqtt
# docker compose up -d

## 1. Compute timestamp
#$ts = Get-Date -UFormat %s

# 2. Run mosquitto_pub inside the Mosquitto container
#docker run --rm --network mqtt-kafka-net eclipse-mosquitto `
#  mosquitto_pub -h mosquitto -t modbus/sensor1 `
#    -m "{\"device_id\":\"sensor1\",\"V_L1\":230,\"I_L1\":5,\"VA_L1\":1150,\"P_L1\":1035,\"timestamp\":$ts}"



#$ts = Get-Date -UFormat %s
#$payload = '{"device_id":"sensor1","V_L1":230,"I_L1":5,"VA_L1":1150,"P_L1":1035,"timestamp":' + $ts + '}'

#docker run --rm --network mqtt-kafka-net eclipse-mosquitto `
  #mosquitto_pub -h mosquitto -t modbus/sensor1 -m $payload


# Fixed Modbus-to-MQTT Forwarder
import time, datetime, struct, signal, sys, json
from pymodbus.client import ModbusTcpClient
import paho.mqtt.client as mqtt

# ─── CONFIG ──────────────────────────────────────────────
MODBUS_HOST   = "localhost"
MODBUS_PORT   = 502
SLAVE_ID      = 1

MQTT_BROKER   = "localhost"
MQTT_PORT     = 1883
MQTT_TOPIC    = "modbus/sensor1"

POLL_INTERVAL = 2  # seconds

# ─── HELPER ───────────────────────────────────────────────
def regs_to_float(high, low):
    try:
        b = high.to_bytes(2, 'big') + low.to_bytes(2, 'big')
        return struct.unpack('>f', b)[0]
    except Exception as e:
        print("Error converting registers to float:", e)
        return None

# ─── SETUP CLIENTS ─────────────────────────────────────────
modbus_client = ModbusTcpClient(MODBUS_HOST, port=MODBUS_PORT)
if not modbus_client.connect():
    print("Cannot connect to Modbus at", MODBUS_HOST, MODBUS_PORT)
    sys.exit(1)

mqtt_client = mqtt.Client()
try:
    mqtt_client.connect(MQTT_BROKER, MQTT_PORT, keepalive=60)
    mqtt_client.loop_start()
except Exception as e:
    print("Failed to connect to MQTT broker:", e)
    sys.exit(1)

stop = False
def on_signal(sig, frame):
    global stop
    stop = True
signal.signal(signal.SIGINT, on_signal)
signal.signal(signal.SIGTERM, on_signal)

print("▶️ Starting forwarder (Modbus → MQTT)...")

while not stop:
    try:
        # Read values from Modbus simulator
        rr_v  = modbus_client.read_holding_registers(address=1,  count=2, slave=SLAVE_ID)
        rr_i  = modbus_client.read_holding_registers(address=13, count=2, slave=SLAVE_ID)
        rr_va = modbus_client.read_holding_registers(address=19, count=2, slave=SLAVE_ID)
        rr_p  = modbus_client.read_holding_registers(address=25, count=2, slave=SLAVE_ID)


        if rr_v.isError() or rr_i.isError() or rr_va.isError() or rr_p.isError():
            print("⚠️ One or more Modbus read errors")
            time.sleep(POLL_INTERVAL)
            continue

        V  = regs_to_float(rr_v.registers[0], rr_v.registers[1])
        I  = regs_to_float(rr_i.registers[0], rr_i.registers[1])
        VA = regs_to_float(rr_va.registers[0], rr_va.registers[1])
        P  = regs_to_float(rr_p.registers[0], rr_p.registers[1])

        payload = {
            "device_id": "sensor1",
            "V_L1": round(V, 2) if V else None,
            "I_L1": round(I, 2) if I else None,
            "VA_L1": round(VA, 2) if VA else None,
            "P_L1": round(P, 2) if P else None,
            "timestamp": int(time.time())

        }

        result = mqtt_client.publish(MQTT_TOPIC, json.dumps(payload))
        status = result[0]
        if status == 0:
            print("Published to MQTT:", payload)
        else:
            print("Failed to publish to MQTT, status:", status)

    except Exception as e:
        print("Unexpected error:", e)

    time.sleep(POLL_INTERVAL)

print("Stopping forwarder...")
modbus_client.close()
mqtt_client.disconnect()


C:\Users\nguye\AppData\Local\Temp\ipykernel_98236\1276579888.py:36: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  mqtt_client = mqtt.Client()


▶️ Starting forwarder (Modbus → MQTT)...
Published to MQTT: {'device_id': 'sensor1', 'V_L1': 220.0, 'I_L1': 5.0, 'VA_L1': 1100.0, 'P_L1': 990.0, 'timestamp': '2025-07-15T09:03:54.990263Z'}
Published to MQTT: {'device_id': 'sensor1', 'V_L1': 230.0, 'I_L1': 6.0, 'VA_L1': 1380.0, 'P_L1': 1242.0, 'timestamp': '2025-07-15T09:03:56.991065Z'}
Published to MQTT: {'device_id': 'sensor1', 'V_L1': 240.0, 'I_L1': 7.0, 'VA_L1': 1680.0, 'P_L1': 1512.0, 'timestamp': '2025-07-15T09:03:58.992888Z'}
Published to MQTT: {'device_id': 'sensor1', 'V_L1': 220.0, 'I_L1': 5.0, 'VA_L1': 1100.0, 'P_L1': 990.0, 'timestamp': '2025-07-15T09:04:00.995144Z'}
Published to MQTT: {'device_id': 'sensor1', 'V_L1': 230.0, 'I_L1': 6.0, 'VA_L1': 1380.0, 'P_L1': 1242.0, 'timestamp': '2025-07-15T09:04:02.997490Z'}
Published to MQTT: {'device_id': 'sensor1', 'V_L1': 240.0, 'I_L1': 7.0, 'VA_L1': 1680.0, 'P_L1': 1512.0, 'timestamp': '2025-07-15T09:04:04.999731Z'}
Published to MQTT: {'device_id': 'sensor1', 'V_L1': 220.0, 'I_L1': 

<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>